# Train transformer

In [65]:
import torch
from torch import nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoTokenizer
from utils.data_utils import AG_NEWS_DATASET
from utils.constants import *
from utils.training import Learner

from quantization.binarize import binarize, IRLinear
from quantization.transformer import Transformer
from quantization.quantize import quantizer
from quantization.pytorch_api import ModelQuant
from quantization.fully_quantize import Model as fullyQuantModel

from utils.train_utils import change_t
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
def create_model(vocab_size, quant_type=None, quant_method=None, bit_num=None, quant_pattern=None):
    '''
    Create training model based on sepcified quant_type
    ----------
    Arguments:
    quant_type    - quant type, should be one of [None, 'quantization', 'binarization']
    quant_method  - quant method to use, if quant_type is None, it should also be None
                    For 'quantization', should be one of ['basic', 'pytorch', 'fully']
                    For 'binarization', should be one of ['basic', 'ir']
    bit_num       - bit number for each parameter, only works when quant_type is 'quantization'
                    should be one of [8,4,2]
    quant_pattern - quantization pattern, should be one of ['MHA', 'FFN', 'CLS', 'ALL']
    '''
    model = Transformer(d_model=BASELINE_MODEL_DIM,
                             d_ff=BASELINE_FFN_DIM,
                             d_hidden=BASELINE_HIDDEN_DIM,
                             h=BASELINE_MODEL_NUMBER_OF_HEADS,
                             n_layers=BASELINE_MODEL_NUMBER_OF_LAYERS,
                             n_class=4,
                             vocab=vocab_size
                            )
    
    __quant_type__ = [None,'quantization','binarization']
    __bit_num__ = [None,8,4,2]
    __quant_pattern__ = [None,'MHA', 'FFN', 'CLS', 'ALL']
    
    assert quant_type in __quant_type__, f"Unimplemented quantization type, should be one of {__quant_type__}, got '{quant_type}'!"
    assert bit_num in __bit_num__, f"Unimplemented bit number, should be one of {__bit_num__}, got '{bit_num}'!"
    assert quant_pattern in __quant_pattern__, f"Unimplemented quantization method, should be one of {__quant_pattern__}, got '{quant_pattern}'!"
    
    if quant_type == None:
        if quant_method is not None:
            print(f"Quant method {quant_method} will not work in baseline model!")
        if bit_num is not None:
            print(f"Bit number {bit_num} will not work in baseline model!")
        if quant_pattern is not None:
            print(f"Quant pattern {quant_pattern} will not work in baseline model!")
    
    elif quant_type == 'quantization':
        __quant_method__ = ['basic', 'pytorch', 'fully']
        
        assert quant_method in __quant_method__, f"Unimplemented quantization method, should be one of {__quant_method__}, got '{quant_method}'!"
        assert bit_num != None, f"Bit number can not be None!"
        assert quant_pattern != None, f"Quant pattern can not be None!"
        
        if quant_method == 'basic':
            if quant_pattern != 'ALL':
                print(f"Current quant method {quant_method} can only quantize the whole network, quant pattern {quant_pattern} will not work!")
            model = quantizer(model, bit_num, True)
            
        elif quant_method == 'pytorch':
            model = ModelQuant(d_model=BASELINE_MODEL_DIM,
                               d_ff=BASELINE_FFN_DIM,
                               d_hidden=BASELINE_HIDDEN_DIM,
                               h=BASELINE_MODEL_NUMBER_OF_HEADS,
                               n_layers=BASELINE_MODEL_NUMBER_OF_LAYERS,
                               n_class=4,
                               vocab=tokenizer.vocab_size,
                               quant_ffn=((quant_pattern == 'FFN')|(quant_pattern == 'ALL')),
                               quant_mha=((quant_pattern == 'MHA')|(quant_pattern == 'ALL')),
                               quant_classifier=((quant_pattern == 'CLS')|(quant_pattern == 'ALL')),
                               bit_num=bit_num)
            
        elif quant_method == 'fully':
            print("For fully_quantized model, bit number and quant pattern will not work!")
            model = fullyQuantModel(4,
                tokenizer.vocab_size,
                BASELINE_MODEL_NUMBER_OF_LAYERS,
                BASELINE_MODEL_NUMBER_OF_HEADS,
                BASELINE_MODEL_DIM)
            
    elif quant_type == 'binarization':
        __quant_method__ = ['basic', 'ir']
        assert quant_method in __quant_method__, f"Unimplemented quantization method, should be one of {__quant_method__}, got '{quant_method}'!"
        assert quant_pattern != None, f"Quant pattern can not be None!"
        print(f"For binarization model, bit num will not work!")
        
        binarize(model, quant_pattern, skip_final=True, qk_only=True)
    
    return model
    

In [8]:
# load dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_dl, test_dl = AG_NEWS_DATASET(tokenizer, batch_size = BATCH_SIZE).load_data()

# create model
model = Transformer(d_model=BASELINE_MODEL_DIM,
                    d_ff=BASELINE_FFN_DIM,
                    d_hidden=BASELINE_HIDDEN_DIM,
                    h=BASELINE_MODEL_NUMBER_OF_HEADS,
                    n_layers=BASELINE_MODEL_NUMBER_OF_LAYERS,
                    n_class=4,
                    vocab=tokenizer.vocab_size
                   )

binarize(model, 'ALL', binarize_layer='ir', skip_final=True, qk_only=False)
print(model)

# loss func
loss_fn = nn.CrossEntropyLoss()

# baseline training config -> do not change!
optim = Adam(model.parameters(), lr= 1e-4)
scheduler = MultiStepLR(optim, milestones=[10,15], gamma=0.1)

train_config ={'model': model,
               'loss_fn': loss_fn,
               'optim': optim,
               'scheduler': scheduler,
               'datasets': [train_dl, test_dl],
               'epochs': 10,
               'batch_size': BATCH_SIZE
               }

train_config['exp_name'] = 'transformer_IRNET'

# training
learner_ag_news = Learner(train_config, ir = True)

Transformer(
  (input_embeddings): Embeddings(
    (token_embedding): Embedding(30522, 512)
    (pos_embedding): Embedding(512, 512)
  )
  (input_encodings): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sublayer_attention): ModuleList(
    (0): sublayerConnectionAttention(
      (multiheads): MultiheadAttention(
        (heads): ModuleList(
          (0): IRLinear (512 -> 512)
          (1): IRLinear (512 -> 512)
          (2): IRLinear (512 -> 512)
        )
        (output): IRLinear (512 -> 512)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layernorm): LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): sublayerConnectionAttention(
      (multiheads): MultiheadAttention(
        (heads): ModuleList(
          (0): IRLinear (512 -> 512)
          (1): IRLinear (512 -> 512)
          (2): IRLinear (512 -> 512)
        )
        (output): IRLinear (512 -> 512)
        (dropout): Dropout(p=0.1, inplace=False)
      )
   

In [9]:
print(model.sublayer_attention[0].multiheads.heads[1].t)

None


In [10]:
learner_ag_news.train()

  0%|                                                    | 0/10 [00:00<?, ?it/s]

current lr 1.00000e-04
tensor([0.1000], device='cuda:0')
Epoch: [0][0/3750]	Loss 21165.4707	Prec@1 31.250
Epoch: [0][100/3750]	Loss 24068.9844	Prec@1 31.374
Epoch: [0][200/3750]	Loss 17295.1633	Prec@1 29.058
Epoch: [0][300/3750]	Loss 14339.7965	Prec@1 28.686
Epoch: [0][400/3750]	Loss 12647.1906	Prec@1 28.265
Epoch: [0][500/3750]	Loss 11378.3273	Prec@1 28.412
Epoch: [0][600/3750]	Loss 10572.3136	Prec@1 28.026
Epoch: [0][700/3750]	Loss 9866.4458	Prec@1 27.960
Epoch: [0][800/3750]	Loss 9347.8048	Prec@1 27.809
Epoch: [0][900/3750]	Loss 8874.2556	Prec@1 27.629
Epoch: [0][1000/3750]	Loss 8416.5995	Prec@1 27.691
Epoch: [0][1100/3750]	Loss 8019.8246	Prec@1 27.725
Epoch: [0][1200/3750]	Loss 7681.0817	Prec@1 27.597
Epoch: [0][1300/3750]	Loss 7355.8397	Prec@1 27.693
Epoch: [0][1400/3750]	Loss 7066.3091	Prec@1 27.663
Epoch: [0][1500/3750]	Loss 6801.2076	Prec@1 27.625
Epoch: [0][1600/3750]	Loss 6558.7160	Prec@1 27.567
Epoch: [0][1700/3750]	Loss 6338.2557	Prec@1 27.504
Epoch: [0][1800/3750]	Loss 611

 10%|████▎                                      | 1/10 [01:49<16:22, 109.21s/it]

current lr 1.00000e-04
tensor([0.2887], device='cuda:0')
Epoch: [1][0/3750]	Loss 585.1575	Prec@1 15.625
Epoch: [1][100/3750]	Loss 2129.6004	Prec@1 32.147
Epoch: [1][200/3750]	Loss 1411.7155	Prec@1 30.193
Epoch: [1][300/3750]	Loss 1134.2423	Prec@1 29.527
Epoch: [1][400/3750]	Loss 982.5048	Prec@1 28.858
Epoch: [1][500/3750]	Loss 908.4314	Prec@1 29.148
Epoch: [1][600/3750]	Loss 883.7329	Prec@1 29.201
Epoch: [1][700/3750]	Loss 833.7551	Prec@1 29.017
Epoch: [1][800/3750]	Loss 807.0178	Prec@1 28.792
Epoch: [1][900/3750]	Loss 778.5893	Prec@1 28.548
Epoch: [1][1000/3750]	Loss 745.7690	Prec@1 28.553
Epoch: [1][1100/3750]	Loss 718.5661	Prec@1 28.590
Epoch: [1][1200/3750]	Loss 698.2553	Prec@1 28.450
Epoch: [1][1300/3750]	Loss 685.3663	Prec@1 28.550
Epoch: [1][1400/3750]	Loss 671.2385	Prec@1 28.437
Epoch: [1][1500/3750]	Loss 652.4644	Prec@1 28.423
Epoch: [1][1600/3750]	Loss 637.8115	Prec@1 28.508
Epoch: [1][1700/3750]	Loss 627.9484	Prec@1 28.516
Epoch: [1][1800/3750]	Loss 617.0398	Prec@1 28.585
Ep

 20%|████████▌                                  | 2/10 [03:36<14:25, 108.20s/it]

...

 90%|██████████████████████████████████████▋    | 9/10 [15:52<01:45, 105.05s/it]

Epoch[8] *Validation*: Prec@1 87.895
current lr 1.00000e-04
tensor([1395.2537], device='cuda:0')
Epoch: [9][0/3750]	Loss 60.4715	Prec@1 84.375
Epoch: [9][100/3750]	Loss 118.5935	Prec@1 86.170
Epoch: [9][200/3750]	Loss 86.3018	Prec@1 88.650
Epoch: [9][300/3750]	Loss 72.8353	Prec@1 89.826
Epoch: [9][400/3750]	Loss 65.0825	Prec@1 90.461
Epoch: [9][500/3750]	Loss 63.0385	Prec@1 90.625
Epoch: [9][600/3750]	Loss 59.2699	Prec@1 90.979
Epoch: [9][700/3750]	Loss 56.0551	Prec@1 91.200
Epoch: [9][800/3750]	Loss 53.2268	Prec@1 91.487
Epoch: [9][900/3750]	Loss 53.1662	Prec@1 91.478
Epoch: [9][1000/3750]	Loss 52.0621	Prec@1 91.574
Epoch: [9][1100/3750]	Loss 50.7428	Prec@1 91.638
Epoch: [9][1200/3750]	Loss 50.8852	Prec@1 91.517
Epoch: [9][1300/3750]	Loss 50.0650	Prec@1 91.593
Epoch: [9][1400/3750]	Loss 49.8631	Prec@1 91.537
Epoch: [9][1500/3750]	Loss 49.8908	Prec@1 91.514
Epoch: [9][1600/3750]	Loss 49.6807	Prec@1 91.511
Epoch: [9][1700/3750]	Loss 49.3541	Prec@1 91.509
Epoch: [9][1800/3750]	Loss 48.96

100%|██████████████████████████████████████████| 10/10 [17:36<00:00, 105.70s/it]

Epoch[9] *Validation*: Prec@1 87.855


[30.894736842105264,
 34.89473684210526,
 54.25,
 78.46052631578948,
 84.44736842105263,
 88.36842105263158,
 87.78947368421052,
 88.42105263157895,
 87.89473684210526,
 87.85526315789474]

# Memory compute

In [67]:
from utils.utils import count_memory_size

In [68]:
count_memory_size(model)

82497552

In [69]:
total = count_memory_size(model) - count_memory_size(model.input_embeddings)
for name, layer in model.named_children():
    print(f'{name}: {count_memory_size(layer)/total}')

input_embeddings: 3.355749760294658
input_encodings: 0.0
sublayer_attention: 0.44420377699589014
sublayer_ffn: 0.4439875142028055
classifier: 0.11180870880130434
